# 1.1 Data

In [ ]:
# Imports
import numpy as np
from glob import glob
from os import path
from scipy.special import comb
from scipy.optimize import minimize
from scipy.stats import binom
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

data_dir = "data"
file_paths = glob(path.join(data_dir, "*.txt"))

# N x M x K
# 5 FILES x 7 ROWS  x 5 COLUMNS
# ROW 1: Audio 
# ROW 2: Visual
# ROW 3-7: Visual going from 'b' (row 3) to 'd' (row 7)
# Columns: Audio from 'b' (col 1) to 'd' (col 5)
data = np.array([np.loadtxt(fname) for fname in file_paths], dtype=np.int64)
N, M, K = data.shape

AUDIO_DATA       = data[:, 0, :]
VISUAL_DATA      = data[:, 1, :]
AUDIOVISUAL_DATA = data[:, 2:, :]

In [ ]:
def baseline_softmax(theta):
    e = np.exp(theta)
    return e / (e +1 )

In [ ]:
def binomial_pmf(k, n, p):
    return comb(n, k) * np.power(p, k) * np.power(1 - p, n - k)

In [ ]:
args = (data[0], 24, np.vstack([p_a.T, p_v.T, p_av]))
np.isclose(binom.logpmf(*args), np.log(binomial_pmf(*args)))

In [ ]:
def objective_function(theta, subject_data):
    theta_a = theta[0:K]
    theta_v = theta[K: ]

    p_a = np.array([baseline_softmax(theta) for theta in theta_a]).reshape(-1,1)
    p_v = np.array([baseline_softmax(theta) for theta in theta_v]).reshape(-1,1)
    
    # Outer product for all combinations
    p_av = (p_a @ p_v.T) / (p_a @ p_v.T + (1 - p_a) @ (1 - p_v).T)
    # likelihoods = binom.logpmf(subject_data, 24, np.vstack([p_a.T, p_v.T, p_av]))
    likelihoods = np.log(binomial_pmf(subject_data, 24, np.vstack([p_a.T, p_v.T, p_av])))

    return -(likelihoods.sum())

theta_set = []
p_set = []
for i in range(5):
    
    theta = np.random.randn(K*2)
    opt_result = minimize(objective_function, theta, args=(data[i]))
    objective, theta_a, theta_v, success = (
        opt_result.fun, 
        (opt_result.x[0:K]), 
        (opt_result.x[K:]), 
        opt_result.success
    )

    theta_set.append(np.concatenate([theta_a,theta_v]))
    print("Subject ", i)
    print("Converged:",success, "NLL:",objective)
    print("theta_a", theta_a)
    print("theta_v", theta_v)
    p_a = np.array([baseline_softmax(theta) for theta in theta_a]).reshape(-1,1)
    p_v = np.array([baseline_softmax(theta) for theta in theta_v]).reshape(-1,1)
    p_av = (p_a @ p_v.T) / (p_a @ p_v.T + (1 - p_a) @ (1 - p_v).T)
    p_set.append(np.vstack([p_a.T, p_v.T, p_av]))
    print("===")
    print("p_a", p_a.flatten())
    print("p_v", p_v.flatten())
    print("p_av\n", np.round(p_av, 3))
    print("===")
    print("data")
    print(np.round(binom.pmf(data[i][2:], 24, p_av), 3))
    print("\n\n")

In [ ]:
x = p_set
y = list(data/24)

fig = plt.figure(figsize = (12,8))
ax1 = fig.add_subplot(111)


for idx in range(5):
    ax1.scatter(x[idx], y[idx],s=160,label=idx+1)
plt.legend(loc='upper left')
plt.title("FLMP response probabilities vs. the response proportions",fontsize=20)
plt.style.use('fivethirtyeight')
plt.show()


In [ ]:
print("\\\\\n".join([f"Subject {i+1} & " + " & ".join(np.char.mod('%.2f', x)) for i, x in enumerate(p_set)]))